In [19]:
%pip install transformers

Looking in indexes: https://nexus.iisys.de/repository/ki-awz-pypi-group/simple, https://pypi.org/simple
Note: you may need to restart the kernel to use updated packages.


In [20]:
%pip install accelerate

Looking in indexes: https://nexus.iisys.de/repository/ki-awz-pypi-group/simple, https://pypi.org/simple
Note: you may need to restart the kernel to use updated packages.


### !! Restart the Kernel !!

In [2]:
import json
import csv
import pandas as pd
import re
from datetime import datetime 
import random
import os
import json

In [18]:
def pick_a_random_template(template_list):
    return random.choice(template_list)
def insert_chart_type(target_string, chart_type):
    res = target_string.replace("{chart_type}", chart_type)
    return res
def insert_chart_title(target_string, chart_title):
    res = target_string.replace("{title}", chart_title)
    return res
#crutch works for the first iteration of my line charts(not universal)
def insert_dates(target_string, dates):
    date_format = random.choice([0, 1])
    dates_in_datetime = [datetime.strptime(date, "%Y-%m-%d") for date in dates]
    dates_to_export = []
    #example of date format swap
    if(date_format == 0):
        dates_to_export = [datetime.strftime(date, "%Y-%m-%d") for date in dates_in_datetime]
    else:
        dates_to_export = [datetime.strftime(date, "%B %d, %Y") for date in dates_in_datetime]
    str_dts = ""
    for date in dates_to_export:
        str_dts = str_dts + date + ", " 
    str_dts = str_dts.strip()
    str_dts = str_dts[:-1]
    formats = []
    res = target_string.replace("{dates}", str_dts)
    return res
def insert_string_xvalues(target_string, xvalues):
    temp = ""
    for xval in xvalues:
        temp = temp + xval + ", "
    temp = temp.strip()
    temp = temp[:-1]
    res = target_string.replace("{xvalues}", temp)
    return res
#crutch works for the first iteration of my line charts(not universal)
def insert_values(target_string, values):
    str_vls = ""
    for val in values:
        str_vls = str_vls + str(val) + ", " 
    str_vls = str_vls.strip()
    str_vls = str_vls[:-1]
    res = target_string.replace("{values}", str_vls)
    return res
def insert_yvalues(target_string, yvalues):
    str_vls = ""
    for val in yvalues:
        str_vls = str_vls + str(val) + ", " 
    str_vls = str_vls.strip()
    str_vls = str_vls[:-1]
    res = target_string.replace("{yvalues}", str_vls)
    return res
def insert_date_value_pairs(target_string, dates, values):
    date_format = random.choice([0, 1])
    dates_in_datetime = [datetime.strptime(date, "%Y-%m-%d") for date in dates]
    dates_to_export = []
    #example of date format swap
    if(date_format == 0):
        dates_to_export = [datetime.strftime(date, "%Y-%m-%d") for date in dates_in_datetime]
    else:
        dates_to_export = [datetime.strftime(date, "%B %d, %Y") for date in dates_in_datetime]
    connecting_words = ["at", "with a value of", "equaling to", "being at"]
    kvpair = ""
    for i in range(0, len(dates_to_export)):
        kvpair = kvpair + dates_to_export[i] + " " + random.choice(connecting_words) + " " + str(values[i]) + "; "
    kvpair = kvpair.strip()
    res = target_string.replace("{date_value_key_pairs}", kvpair)
    return res
def insert_x_y_value_pairs(target_string, xvalues, yvalues):
    connecting_words = ["at", "with a value of", "equaling to", "being at"]
    kvpair = ""
    for i in range(0, len(xvalues)):
        kvpair = kvpair + xvalues[i] + " " + random.choice(connecting_words) + " " + str(yvalues[i]) + "; "
    kvpair = kvpair.strip()
    res = target_string.replace("{xvalue_yvalue_key_pairs}", kvpair)
    return res
def insert_all(target_string, chart_type, chart_title, dates, values):
    res = insert_chart_type(target_string, chart_type)
    res = insert_chart_title(res, chart_title)
    if "{date_value_key_pairs}" in target_string:
        res = insert_date_value_pairs(res, dates, values)
    else:
        res = insert_dates(res, dates)
        res = insert_values(res, values)
    return res
def insert_all_universal(target_string, chart_type, chart_title, xvalues, yvalues):
    res = insert_chart_type(target_string, chart_type)
    res = insert_chart_title(res, chart_title)
    if "{xvalue_yvalue_key_pairs}" in target_string:
        res = insert_x_y_value_pairs(res, xvalues, yvalues)
    else:
        res = insert_string_xvalues(res, xvalues)
        res = insert_yvalues(res, yvalues)
    return res
def synonym_generator(target_string):
    prompt = target_string
    synonyms = {
    "Generate": ["Create", "Produce", "Construct", "Craft", "Make", "Generate"],
    "with": ["utilizing", "using", "with"],
    "chart": ["graph", "diagram", "plot", "chart"],
    "titled": ["named", "called", "identified as", "titled"],
    "calling it": ["naming it", "identifying it as", "calling it"],
    "will": ["is going to", "will"],
    "display": ["show", "showcase", "represent", "display"]
    }
    for word, syn_list in synonyms.items():
        if word in prompt:
            synonym = random.choice(syn_list)
        else:
            #print(syn_list)
            for syn in syn_list:
                if syn in prompt:
                    word = syn
                    synonym = random.choice(syn_list)
                    break
        prompt = prompt.replace(word, synonym)
    return prompt

## NBA DATASET 

In [19]:
df_nba = pd.read_csv('NBA/output.csv')
templates_bar_charts_nba = [
    "Create a {chart_type} chart using ChartJS. The chart should be titled '{title}'. It will display data points for players: {xvalues}. The corresponding statistics for these players are {yvalues} respectively.",
    "Craft a visual representation with ChartJS, ensuring the title reads '{title}'. This chart will showcase the top players and their respective statistics: {xvalue_yvalue_key_pairs}.",
    "Generate a {chart_type} chart using ChartJS. Name it '{title}' and display the performance data for the players: {xvalues}. The associated statistics are as follows: {yvalues}.",
    "Design a {chart_type} chart using ChartJS titled '{title}'. This chart will represent the performance metrics for specific players: {xvalues}. The corresponding values are {yvalues}.",
    "Produce a visual representation using ChartJS, with the title '{title}'. The chart will highlight player performance data: {xvalue_yvalue_key_pairs}.",
    "Construct a {chart_type} chart using ChartJS named '{title}'. This visualization will display the player statistics for: {xvalues}. The associated figures are {yvalues}, respectively.",
    "Create a {chart_type} chart using ChartJS titled '{title}'. It will showcase player performance markers: {xvalues}. The corresponding values are as follows: {yvalues}.",
    "Build a {chart_type} chart with ChartJS, titled '{title}'. This chart will show the statistics for players: {xvalues}. The associated values are {yvalues} respectively."
]
templates_pie_charts_nba = [
    "Create a {chart_type} chart using ChartJS. Title it '{title}'. The chart will show the percentage distribution among players: {xvalues}. Their respective values are {yvalues}.",
    "Design a visual representation with ChartJS titled '{title}'. This chart will display the share of each player in the dataset: {xvalue_yvalue_key_pairs}.",
    "Generate a {chart_type} chart with ChartJS named '{title}'. The chart will depict the percentage breakdown for players: {xvalues}. Their values are {yvalues}.",
    "Craft a {chart_type} chart using ChartJS, titled '{title}'. This visualization will illustrate the proportion of each player's statistics: {xvalues}. The corresponding values are {yvalues}.",
    "Create a {chart_type} chart using ChartJS with the title '{title}'. It will show the distribution of values among players: {xvalue_yvalue_key_pairs}.",
    "Construct a {chart_type} chart with ChartJS titled '{title}'. This chart will present the percentage of each player in the dataset: {xvalues}. Their values are {yvalues}.",
    "Build a {chart_type} chart using ChartJS named '{title}'. The chart will illustrate the distribution among players: {xvalues}. Their respective values are {yvalues}.",
    "Produce a {chart_type} chart using ChartJS with the title '{title}'. This chart will display the percentage share for players: {xvalues}. Their associated values are {yvalues}."
]
templates_line_charts_nba = [
    "Create a {chart_type} chart using ChartJS. Title it '{title}'. This chart will display performance over time for players: {xvalues}. Their corresponding values are {yvalues}.",
    "Design a visual representation with ChartJS, titled '{title}'. This chart will show the trend of player statistics over a period: {xvalue_yvalue_key_pairs}.",
    "Generate a {chart_type} chart using ChartJS named '{title}'. It will illustrate the changes in player performance over time: {xvalues}. The values are {yvalues}.",
    "Craft a {chart_type} chart with ChartJS, titled '{title}'. This visualization will depict the performance trend for players: {xvalues}. The corresponding values are {yvalues}.",
    "Create a {chart_type} chart using ChartJS with the title '{title}'. It will show the performance data over time for players: {xvalue_yvalue_key_pairs}.",
    "Construct a {chart_type} chart with ChartJS titled '{title}'. This chart will present the performance trend for specific players: {xvalues}. The associated values are {yvalues}.",
    "Build a {chart_type} chart using ChartJS named '{title}'. This chart will display the trend of player statistics over time: {xvalues}. Their values are {yvalues}.",
    "Produce a {chart_type} chart using ChartJS with the title '{title}'. This chart will illustrate the time-based performance for players: {xvalues}. Their associated values are {yvalues}."
]

In [20]:
nat_data = []
charts_cfg = df_nba.loc[:, 'Chart Configuration']
for chr in charts_cfg:
    # Chart Type
    chart = 'not found'
    pattern_chart_type = r'"type":\s*\"([^\"]*)'
    match_chart = re.search(pattern_chart_type, chr)
    if match_chart:
        type_value = match_chart.group(1)
        chart = type_value
        print("Extracted type value:", type_value)
    else:
        print("Type value not found.")

    # Title
    title = 'no title'
    pattern_chart_title = r'"label":\s*\"([^\"]*)'
    match_title = re.search(pattern_chart_title, chr)
    if match_title:
        type_value = match_title.group(1)
        title = type_value
        print("Extracted type value:", type_value)
    else:
        print("Type value not found.")

    # Labels
    pattern_labels = r'"labels":\s*(\[[^\]]*\])'
    matches = re.findall(pattern_labels, chr)
    
    for match in matches:
        labels_string = match.replace('\\"', '"')
        array_string = labels_string.replace("'", '"')
        labels_list_names = json.loads(labels_string)
        # print("Extracted labels:", labels_list_names)


    # Data
    pattern_data = r'"data":\s*(\[[^\]]*\])'
    match = re.search(pattern_data, chr)
    
    if match:
        labels_string = match.group(1)
        array_string = labels_string.replace("'", '"')
        labels_list_values = json.loads(array_string)
    else:
        print("Datas not found.")

    print(chart, title, labels_list_names, labels_list_values)
    if chart == "bar":
        natural_language_instance_charts = insert_all_universal(pick_a_random_template(templates_bar_charts_nba), chart, title, labels_list_names, labels_list_values)
    elif chart == "pie":
        natural_language_instance_charts = insert_all_universal(pick_a_random_template(templates_pie_charts_nba), chart, title, labels_list_names, labels_list_values)
    elif chart == "line":
        natural_language_instance_charts = insert_all_universal(pick_a_random_template(templates_line_charts_nba), chart, title, labels_list_names, labels_list_values)

    nat_data.append([natural_language_instance_charts])

nat_data_df = pd.DataFrame(nat_data, columns=['Chart in Natural language'])
nat_data_path = 'NBA/nat_data.csv'
nat_data_df.to_csv(nat_data_path, index=False)
# # Check if the file exists
# if not os.path.isfile("NBA/nat_data.csv"):
#     # File does not exist, write the data with header
#     data.to_csv("NBA/nat_data.csv", mode='w', header=True, index=False)
# else:
#     # File exists, append the data without writing the header
#     data.to_csv("NBA/nat_data.csv", mode='a', header=False, index=False)


Extracted type value: bar
Extracted type value: NBA' Players Points
bar NBA' Players Points ['Iman Shumpert', 'Marcus Camby', 'Antoine Walker'] [4.2, 12.1, 17.5]
Extracted type value: bar
Extracted type value: NBA' Players Assist Percentage
bar NBA' Players Assist Percentage ['Lavoy Allen', 'Rasheed Wallace', 'Damion Lee'] [0.093, 0.085, 0.139]
Extracted type value: bar
Extracted type value: NBA' Players Assists
bar NBA' Players Assists ['Stromile Swift', 'Chris Webber', 'Sekou Doumbouya'] [0.4, 4.6, 0.8]
Extracted type value: bar
Extracted type value: NBA' Players Assists
bar NBA' Players Assists ['Walter McCarty', 'Steve Nash', 'Joe Kleine', 'Dante Cunningham'] [0.7, 5.7, 0.4, 1.0]
Extracted type value: bar
Extracted type value: NBA' Players Assist Percentage
bar NBA' Players Assist Percentage ['Shawn Bradley', 'Jeremiah Martin', 'Beno Udrih', 'DeMarcus Cousins'] [0.04, 0.091, 0.239, 0.154]
Extracted type value: bar
Extracted type value: NBA' Players Defensive Rebound Percentage
bar 

## Movie Dataset

In [21]:
df_movie = pd.read_csv('Movie/output.csv')
templates_bar_charts_movie = [
    "Create a {chart_type} chart using ChartJS. The chart should be titled '{title}'. It will display data points for movies: {xvalues}. The corresponding values are {yvalues} respectively.",
    "Craft a visual representation with ChartJS, ensuring the title reads '{title}'. This chart will showcase different movies and their respective values: {xvalue_yvalue_key_pairs}.",
    "Generate a {chart_type} chart using ChartJS. Name it '{title}' and display the data for the movies: {xvalues}. The associated values are as follows: {yvalues}.",
    "Design a {chart_type} chart using ChartJS titled '{title}'. This chart will represent the metrics for specific movies: {xvalues}. The corresponding values are {yvalues}.",
    "Produce a visual representation using ChartJS, with the title '{title}'. The chart will highlight movie data: {xvalue_yvalue_key_pairs}.",
    "Construct a {chart_type} chart using ChartJS named '{title}'. This visualization will display the statistics for movies: {xvalues}. The associated figures are {yvalues}, respectively.",
    "Create a {chart_type} chart using ChartJS titled '{title}'. It will showcase movie metrics: {xvalues}. The corresponding values are as follows: {yvalues}.",
    "Build a {chart_type} chart with ChartJS, titled '{title}'. This chart will show the data for movies: {xvalues}. The associated values are {yvalues} respectively."
]
templates_pie_charts_movie = [
    "Create a {chart_type} chart using ChartJS. Title it '{title}'. The chart will show the percentage distribution among movies: {xvalues}. Their respective values are {yvalues}.",
    "Design a visual representation with ChartJS titled '{title}'. This chart will display the share of each movie: {xvalue_yvalue_key_pairs}.",
    "Generate a {chart_type} chart with ChartJS named '{title}'. The chart will depict the percentage breakdown for movies: {xvalues}. Their values are {yvalues}.",
    "Craft a {chart_type} chart using ChartJS, titled '{title}'. This visualization will illustrate the proportion of each movie's metrics: {xvalues}. The corresponding values are {yvalues}.",
    "Create a {chart_type} chart using ChartJS with the title '{title}'. It will show the distribution of values among movies: {xvalue_yvalue_key_pairs}.",
    "Construct a {chart_type} chart with ChartJS titled '{title}'. This chart will present the percentage of each movie: {xvalues}. Their values are {yvalues}.",
    "Build a {chart_type} chart using ChartJS named '{title}'. The chart will illustrate the distribution among movies: {xvalues}. Their respective values are {yvalues}.",
    "Produce a {chart_type} chart using ChartJS with the title '{title}'. This chart will display the percentage share for movies: {xvalues}. Their associated values are {yvalues}."
]
templates_line_charts_movie = [
    "Create a {chart_type} chart using ChartJS. Title it '{title}'. This chart will display the trend over time for movies: {xvalues}. Their corresponding values are {yvalues}.",
    "Design a visual representation with ChartJS, titled '{title}'. This chart will show the trend of movie metrics over a period: {xvalue_yvalue_key_pairs}.",
    "Generate a {chart_type} chart using ChartJS named '{title}'. It will illustrate the changes in movie data over time: {xvalues}. The values are {yvalues}.",
    "Craft a {chart_type} chart with ChartJS, titled '{title}'. This visualization will depict the trend for movies: {xvalues}. The corresponding values are {yvalues}.",
    "Create a {chart_type} chart using ChartJS with the title '{title}'. It will show the data over time for movies: {xvalue_yvalue_key_pairs}.",
    "Construct a {chart_type} chart with ChartJS titled '{title}'. This chart will present the trend for specific movies: {xvalues}. The associated values are {yvalues}.",
    "Build a {chart_type} chart using ChartJS named '{title}'. This chart will display the trend of movie metrics over time: {xvalues}. Their values are {yvalues}.",
    "Produce a {chart_type} chart using ChartJS with the title '{title}'. This chart will illustrate the time-based trend for movies: {xvalues}. Their associated values are {yvalues}."
]

In [22]:
nat_data = []
charts_cfg = df_movie.loc[:, 'Chart Configuration']
for chr in charts_cfg:
    # Chart Type
    chart = 'not found'
    pattern_chart_type = r'"type":\s*\"([^\"]*)'
    match_chart = re.search(pattern_chart_type, chr)
    if match_chart:
        type_value = match_chart.group(1)
        chart = type_value
        # print("Extracted type value:", type_value)
    else:
        print("Type value not found.")

    # Title
    title = 'no title'
    pattern_chart_title = r'"label":\s*\"([^\"]*)'
    match_title = re.search(pattern_chart_title, chr)
    if match_title:
        type_value = match_title.group(1)
        title = type_value
        # print("Extracted type value:", type_value)
    else:
        print("Type value not found.")

    # Labels
    pattern_labels = r'"labels":\s*(\[[^\]]*\])'
    matches = re.findall(pattern_labels, chr)
    
    for match in matches:
        labels_string = match.replace('\\"', '"')
        array_string = labels_string.replace("'", '"')
        labels_list_names = json.loads(labels_string)
        # print("Extracted labels:", labels_list_names)

    # Data
    pattern_data = r'"data":\s*(\[[^\]]*\])'
    match = re.search(pattern_data, chr)
    
    if match:
        labels_string = match.group(1)
        array_string = labels_string.replace("'", '"')
        labels_list_values = json.loads(array_string)
        # print("Extracted data:", labels_list_values)
    else:
        print("Datas not found.")

    print(chart, title, labels_list_names, labels_list_values)
    if chart == "bar":
        natural_language_instance_charts = insert_all_universal(pick_a_random_template(templates_bar_charts_movie), chart, title, labels_list_names, labels_list_values)
    elif chart == "pie":
        natural_language_instance_charts = insert_all_universal(pick_a_random_template(templates_pie_charts_movie), chart, title, labels_list_names, labels_list_values)
    elif chart == "line":
        natural_language_instance_charts = insert_all_universal(pick_a_random_template(templates_line_charts_movie), chart, title, labels_list_names, labels_list_values)

    nat_data.append([natural_language_instance_charts])
    #nat_data.append([natural_language_example])

nat_data_df = pd.DataFrame(nat_data, columns=['Chart in Natural language'])
nat_data_path = 'Movie/nat_data.csv'
nat_data_df.to_csv(nat_data_path, index=False)


bar Movie Theaters ['Avatar', 'Planet of the Apes', "The General's Daughter"] [3452, 3500, 2856]
bar Movie Theaters ['Grown Ups 2', "He's Just Not That Into You", 'Underworld'] [3491, 3175, 2915]
bar Movie Opening ['Paranormal Activity 3', '2 Guns', 'Sex and the City 2'] [52568183, 27059130, 31001870]
bar Movie Opening ['Deadpool 2', 'Avatar: The Way of Water', 'Christopher Robin', 'The Firm'] [125507153, 134100226, 24585139, 25400000]
bar Movie Total gross ['Apollo 13', 'The Haunting in Connecticut', 'Dungeons & Dragons: Honor Among Thieves', 'Fantastic Four'] [172071312, 55389516, 93277026, 56117548]
bar Movie Total gross ['S.W.A.T.', 'What Women Want', 'Doctor Strange in the Multiverse of Madness', 'Sin City'] [116934650, 182811707, 411331607, 74103820]
bar Movie Theaters ['Thor', 'San Andreas', 'Tangled', 'A Christmas Carol', 'Mulan'] [3955, 3777, 3603, 3683, 2888]
bar Movie Opening ['Once Upon a Time in Mexico', 'The Campaign', 'Alien vs. Predator', 'The Suicide Squad', 'The Last 

## Cars Dataset

In [23]:
df_cars = pd.read_csv('Cars/output.csv')
templates_bar_charts_cars = [
    "Create a {chart_type} chart using ChartJS. The chart should be titled '{title}'. It will display data points for car models: {xvalues}. The corresponding values are {yvalues} respectively.",
    "Craft a visual representation with ChartJS, ensuring the title reads '{title}'. This chart will showcase different car models and their respective values: {xvalue_yvalue_key_pairs}.",
    "Generate a {chart_type} chart using ChartJS. Name it '{title}' and display the data for the car models: {xvalues}. The associated values are as follows: {yvalues}.",
    "Design a {chart_type} chart using ChartJS titled '{title}'. This chart will represent the metrics for specific car models: {xvalues}. The corresponding values are {yvalues}.",
    "Produce a visual representation using ChartJS, with the title '{title}'. The chart will highlight car model data: {xvalue_yvalue_key_pairs}.",
    "Construct a {chart_type} chart using ChartJS named '{title}'. This visualization will display the statistics for car models: {xvalues}. The associated figures are {yvalues}, respectively.",
    "Create a {chart_type} chart using ChartJS titled '{title}'. It will showcase car model metrics: {xvalues}. The corresponding values are as follows: {yvalues}.",
    "Build a {chart_type} chart with ChartJS, titled '{title}'. This chart will show the data for car models: {xvalues}. The associated values are {yvalues} respectively."
]
templates_pie_charts_cars = [
    "Create a {chart_type} chart using ChartJS. Title it '{title}'. The chart will show the percentage distribution among car models: {xvalues}. Their respective values are {yvalues}.",
    "Design a visual representation with ChartJS titled '{title}'. This chart will display the share of each car model: {xvalue_yvalue_key_pairs}.",
    "Generate a {chart_type} chart with ChartJS named '{title}'. The chart will depict the percentage breakdown for car models: {xvalues}. Their values are {yvalues}.",
    "Craft a {chart_type} chart using ChartJS, titled '{title}'. This visualization will illustrate the proportion of each car model's metrics: {xvalues}. The corresponding values are {yvalues}.",
    "Create a {chart_type} chart using ChartJS with the title '{title}'. It will show the distribution of values among car models: {xvalue_yvalue_key_pairs}.",
    "Construct a {chart_type} chart with ChartJS titled '{title}'. This chart will present the percentage of each car model: {xvalues}. Their values are {yvalues}.",
    "Build a {chart_type} chart using ChartJS named '{title}'. The chart will illustrate the distribution among car models: {xvalues}. Their respective values are {yvalues}.",
    "Produce a {chart_type} chart using ChartJS with the title '{title}'. This chart will display the percentage share for car models: {xvalues}. Their associated values are {yvalues}."
]
templates_line_charts_cars = [
    "Create a {chart_type} chart using ChartJS. Title it '{title}'. This chart will display the trend over time for car models: {xvalues}. Their corresponding values are {yvalues}.",
    "Design a visual representation with ChartJS, titled '{title}'. This chart will show the trend of car model metrics over a period: {xvalue_yvalue_key_pairs}.",
    "Generate a {chart_type} chart using ChartJS named '{title}'. It will illustrate the changes in car model data over time: {xvalues}. The values are {yvalues}.",
    "Craft a {chart_type} chart with ChartJS, titled '{title}'. This visualization will depict the trend for car models: {xvalues}. The corresponding values are {yvalues}.",
    "Create a {chart_type} chart using ChartJS with the title '{title}'. It will show the data over time for car models: {xvalue_yvalue_key_pairs}.",
    "Construct a {chart_type} chart with ChartJS titled '{title}'. This chart will present the trend for specific car models: {xvalues}. The associated values are {yvalues}.",
    "Build a {chart_type} chart using ChartJS named '{title}'. This chart will display the trend of car model metrics over time: {xvalues}. Their values are {yvalues}.",
    "Produce a {chart_type} chart using ChartJS with the title '{title}'. This chart will illustrate the time-based trend for car models: {xvalues}. Their associated values are {yvalues}."
]

In [24]:
nat_data = []
charts_cfg = df_cars.loc[:, 'Chart Configuration']
for chr in charts_cfg:
    # Chart Type
    chart = 'not found'
    pattern_chart_type = r'"type":\s*\"([^\"]*)'
    match_chart = re.search(pattern_chart_type, chr)
    if match_chart:
        type_value = match_chart.group(1)
        chart = type_value
        # print("Extracted type value:", type_value)
    else:
        print("Type value not found.")

    # Title
    title = 'no title'
    pattern_chart_title = r'"label":\s*\"([^\"]*)'
    match_title = re.search(pattern_chart_title, chr)
    if match_title:
        type_value = match_title.group(1)
        title = type_value
        # print("Extracted type value:", type_value)
    else:
        print("Type value not found.")

    # Labels
    pattern_labels = r'"labels":\s*(\[[^\]]*\])'
    matches = re.findall(pattern_labels, chr)
    
    for match in matches:
        labels_string = match.replace('\\"', '"')
        array_string = labels_string.replace("'", '"')
        labels_list_names = json.loads(labels_string)
        # print("Extracted labels:", labels_list_names)

    # Data
    pattern_data = r'"data":\s*(\[[^\]]*\])'
    match = re.search(pattern_data, chr)
    
    if match:
        labels_string = match.group(1)
        array_string = labels_string.replace("'", '"')
        labels_list_values = json.loads(array_string)
        # print("Extracted data:", labels_list_values)
    else:
        print("Datas not found.")

    print(chart, title, labels_list_names, labels_list_values)
    if chart == "bar":
        natural_language_instance_charts = insert_all_universal(pick_a_random_template(templates_bar_charts_cars), chart, title, labels_list_names, labels_list_values)
    elif chart == "pie":
        natural_language_instance_charts = insert_all_universal(pick_a_random_template(templates_pie_charts_cars), chart, title, labels_list_names, labels_list_values)
    elif chart == "line":
        natural_language_instance_charts = insert_all_universal(pick_a_random_template(templates_line_charts_cars), chart, title, labels_list_names, labels_list_values)

    nat_data.append([natural_language_instance_charts])

nat_data_df = pd.DataFrame(nat_data, columns=['Chart in Natural language'])
nat_data_path = 'Cars/nat_data.csv'
nat_data_df.to_csv(nat_data_path, index=False)

bar Cars' Horsepower ['datsun 310 gx', 'plymouth fury iii', 'ford country squire (sw)'] [67.0, 215.0, 142.0]
bar Cars' Acceleration (time to speed) ['honda civic 1300', 'audi 100 ls', 'pontiac safari (sw)'] [16.1, 14.5, 12.0]
bar Cars' Weight (lbs) ['plymouth sapporo', 'datsun 710', 'mercury zephyr'] [2745, 2003, 3070]
bar Cars' Acceleration (time to speed) ['pontiac j2000 se hatchback', 'amc matador', 'plymouth cricket', 'volvo 244dl'] [16.2, 15.5, 20.5, 14.5]
bar Cars' Weight (lbs) ['plymouth horizon', 'plymouth volare premier v8', 'honda civic 1500 gl', 'volkswagen type 3'] [2200, 3940, 1850, 2254]
bar Cars' Acceleration (time to speed) ['plymouth volare', 'ford gran torino (sw)', 'toyota corolla', 'pontiac phoenix lj'] [17.2, 16.0, 16.9, 19.2]
bar Cars' Acceleration (time to speed) ['plymouth reliant', 'toyota corona', 'dodge aspen', 'chevrolet chevette', 'ford gran torino (sw)'] [15.7, 14.2, 18.7, 16.5, 16.0]
bar Cars' Horsepower ['buick skylark', 'chevrolet malibu classic (sw)', 